# <center> Enhanced Tornado Server Example
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 

This notebook is slightly different in each block is a complete server. They run asyncronously, which means only 1 block can run at a time. To run another block, the Python Kernel must be shut down. Therefore, the notebook displays the bad practice of copy and pasted code.

Start wireshark & listen on port 9100 (optionally view filter 'http' protocol) to watch traffic.

In [2]:
import tornado.ioloop
import tornado.web
import time
import json
import os
import base64
import logging

# SimpleClient
Basic functionality -- accept a POST request, and return the original "cipher_text" (which is not encrypted) as "plaintext"

In [3]:
# No encryption

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.write(
            {'message': "The current time is {}".format(time.asctime())}
        )

    def post(self):
        data = json.loads(self.request.body.decode('utf-8'))
        self.write({'original_text': data['cipher_text'], 'plaintext': data['cipher_text']})


# SymmetricKeyClient
Server will respond to a GET request with a symmetric key.  Client then uses this key to encrypt
a message and POST it to the server.  Server decrypts the message with the symmetric key and returns
cipher_text and plaintext back to the client.

**VERY BAD PRACTICE** as symmetric key can be snooped.  No security here at all.

In [4]:
# Symmetric encryption with key exchange via GET.  Bad practice -- don't do this!

from cryptography.fernet import Fernet
SYMMETRIC_KEY = Fernet.generate_key()
f = Fernet(SYMMETRIC_KEY)

class EncryptedHandler(tornado.web.RequestHandler):
    def get(self):
        #Write the key data out
        # Don't ever do this. It is bad. You'll expose the key!
        self.write({'key': SYMMETRIC_KEY.decode('utf-8')})

    def post(self):
        data = json.loads(self.request.body.decode('utf-8'))
        logging.debug(f'Got JSON data: {data}')
        cipher_text = data['cipher_text']
        f = Fernet(SYMMETRIC_KEY)
        plaintext = f.decrypt(cipher_text.encode('utf-8')).decode('utf-8')
        self.write({'ciphertext': cipher_text, 'plaintext': plaintext})

# PublicKeyClient
Server responds to a GET request with its public key.  Client then uses this public key to encrypt a message.  Server uses its private key to decrypt, returning both the original cipher_text and the decrypted plaintext.

This is reasonable practice, as only the server knows its private key, and thus only the server can decrypt, so message privace is maintained.  It is subject to impersonation of the client, though...

It is also potentially slow, as asymmetric encryption is significantly slower than symmetric encryption.

In [5]:
# Asymmetric encryption with the server's public key, which is queried in 
# the initial GET.  This provides message privacy in a reasonable way.

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

PRIVATE_KEY = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
PUBLIC_KEY = PRIVATE_KEY.public_key()

class ServerPublicKeyHandler(tornado.web.RequestHandler):

    def get(self):
        # provide the public key.  This is not bad...  
        # It is, after all, the public key
        pem = PUBLIC_KEY.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
        self.write({'publicKey': pem.decode('ASCII')})

    def post(self):
        data = json.loads(self.request.body.decode('utf-8'))
        cyphertext = data['cipher_text']
        logging.debug(f"Cyphertext: {cyphertext}")
        cypherbytes = base64.b64decode(cyphertext)
        logging.debug(f"Cypherbytes: {cypherbytes}")

        try:
            plaintext = PRIVATE_KEY.decrypt(
                cypherbytes,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
        except ValueError as e:
            logging.warning(f"Value Error: {e}")
            plaintext = None
        self.write({'original_text': data['cipher_text'], 
                    'plaintext': plaintext.decode('utf-8')})

# KeyExchangeClient
Client sends its public key as a URL argument in the GET.  Server replies to the get by encrypting a symmetric key with the client public key.  Client uses its private key to decrypt the symmetric key, and sends subsequent messages encryped with the symmetric key.

Much faster than using asymmetric encryption for the entire conversation.

Still subject to impersonation -- this time on the server side.  Can be cracked by a Person-in-the-middle (PITM) attack.

In [6]:
# Transfer a symmetric key -- kinda like the first example.  But this example
# uses the client's public key to encrypt the symmetric key, so it is safe from snooping.

class ServerPublicKeyExchangeHandler(tornado.web.RequestHandler):

    def get(self):
        encodedClientPublicKey = self.request.query_arguments['client_publicKey'][0]
        clientPublicKeyBytes = base64.b64decode(encodedClientPublicKey)
        clientPublicKey = serialization.load_der_public_key(clientPublicKeyBytes, backend=default_backend())
        # provide the symmetric key, but encrypt it with the client public key 
        # to keep it secret
        logging.debug(f"Symmetric key: {SYMMETRIC_KEY}")
        encryptedSymmetricKey = clientPublicKey.encrypt(SYMMETRIC_KEY,
                                                        padding.OAEP(
                                                            mgf=padding.MGF1(algorithm=hashes.SHA256()),
                                                            algorithm=hashes.SHA256(),
                                                            label=None)
                                                        )
        encodedEncryptedSymmetricKey = base64.b64encode(encryptedSymmetricKey).decode('utf-8')
        self.write({'encodedEncryptedSymmetricKey': encodedEncryptedSymmetricKey})

    def post(self):
        data = json.loads(self.request.body.decode('utf-8'))
        cyphertext = data['cipher_text']
        logging.debug(f"Cyphertext: {cyphertext}")
        cypherbytes = base64.b64decode(cyphertext)
        logging.debug(f"Cypherbytes: {cypherbytes}")

        try:
            plaintext = PRIVATE_KEY.decrypt(
                cypherbytes,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
        except ValueError as e:
            logging.warning(f"Value Error: {e}")
            plaintext = None
        self.write({'original_text': data['cipher_text'], 'plaintext': plaintext.decode('utf-8')})

In [7]:
try:
    logging.basicConfig(level=logging.DEBUG)
    port = 9100
    app = tornado.web.Application([
        (r"/", MainHandler),
        (r"/encrypted/", EncryptedHandler),
        (r"/serverPublicKey/", ServerPublicKeyHandler),
        (r"/serverPublicKeySymmetricKeyExchange/", ServerPublicKeyExchangeHandler)
       ], debug = True) #turn off debugging for production

    app.listen(port)
    print("Listening on port {}".format(port))
    # tornado.ioloop.IOLoop.current().start()
    #Restart Kernel to stop
except OSError:
    os._exit(00)
    #Be sure to run all

Listening on port 9100


INFO:tornado.access:200 POST / (127.0.0.1) 1.00ms
INFO:tornado.access:200 GET /encrypted/ (::1) 0.00ms
DEBUG:root:Got JSON data: {'cipher_text': 'gAAAAABoEp6ihIRnm5PnnfQp62ZJ_GT2HA7sjORdHvZfFG5vqDhkgzQC4u9s0Pq1cGzGxHDfmSZl8ioxATnlgryNrmxsCCk0o6xTNdyMud90tInSGxcVyXO7WYRtDrUiD_qmoLIND1z3JaoPLlypWv2fMg8TNiBTjNGlFxXUSkTe6d6pw0US78b0hBLEooBxjAIaM8SABdNMopbXBjiewC0iAT58QiCpuMjVwD1B4CgIn0KpiX9P2AAx8oDYEo54bulh6eDI7arGueMoMK9yXZlCv5ZPJ4BpSajiqzxUi8Lg-gX9ozTSuPA='}
INFO:tornado.access:200 POST /encrypted/ (::1) 1.00ms
INFO:tornado.access:200 GET /serverPublicKey/ (127.0.0.1) 1.00ms
DEBUG:root:Cyphertext: EOWAJ4ui5qzh99i+aoEQ022vbDRbbHfGtTVSJon0YLybqDF7fVka4iwrI7emHYfPOU6MjHIAj0h8BaKLl2g7iWOjxdTHXzz9ukhgnel9d/irNnzoEgND8/dMiGwCPUeNSwAlWtf9GPAJT5NbDHxA4cjoBKmNB2+OLJnMs0xr1sRj2gTNLYJF49CHAdJnysbYjCpTv5X+2OO2tQRY1tJeXB9zzN/QRC6eJfgM2VeIAdsIzEc4BeBx+OHlmLzDXuK71TSWQjFrkTFrXerMNQWmv5AIPpabkqLWJOJmvER2Mt3CbCmM34BXYPuR3QyF9SCY9b/ogJPKAKN+6hsN5XfbAA==
DEBUG:root:Cypherbytes: b"\x10\xe5\x80'\x8b\xa2\xe6\